<b> Fast food isochrone </b>

In [121]:
#main libraries
import osmnx as ox
import networkx as nx
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point,LineString,Polygon
from matplotlib.colors import ListedColormap

#additional libraries
import matplotlib.cm as cm
import pandas as pd
from geopy.geocoders import Nominatim

#webmap libraries
import folium
from folium.plugins import MiniMap
from folium.plugins import MarkerCluster
from folium.features import CustomIcon
import branca.colormap as comap
from folium.plugins import FloatImage

from folium import FeatureGroup

#use a local cache to save/retrieve http responses instead of calling API repetitively for the same request URL
ox.config(use_cache=True)

In [2]:
def get_latlong(place):
    geolocator = Nominatim(user_agent="specify_your_app_name_here")
    location = geolocator.geocode(place)
    long,lat = location.longitude,location.latitude
    return (lat,long)

def get_network_from_point(location_point,distance,network_type):
    network = ox.graph_from_point(location_point, distance=distance, distance_type='network',network_type=network_type)
    return network

def get_nearest_node(network,yx):
    center_node = ox.get_nearest_node(network,yx)
    return center_node

def make_iso_polys(G, center_node,edge_buff=25, node_buff=50, infill=False):
    
    #adapted from: https://geoffboeing.com/2017/08/isochrone-maps-osmnx-python/ 
    
    isochrone_polys = []
    for trip_time in sorted(trip_times, reverse=True):
        subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='time')

        node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
        nodes_gdf = gpd.GeoDataFrame({'id': subgraph.nodes()}, geometry=node_points)
        nodes_gdf = nodes_gdf.set_index('id')

        edge_lines = []
        for n_fr, n_to in subgraph.edges():
            f = nodes_gdf.loc[n_fr].geometry
            t = nodes_gdf.loc[n_to].geometry
            edge_lines.append(LineString([f,t]))

        n = nodes_gdf.buffer(node_buff).geometry
        e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
        all_gs = list(n) + list(e)
        new_iso = gpd.GeoSeries(all_gs).unary_union
        
        # try to fill in surrounded areas so shapes will appear solid and blocks without white space inside them
        if infill:
            new_iso = Polygon(new_iso.exterior)
        isochrone_polys.append(new_iso)
    return isochrone_polys

In [3]:
#setup travel parameters
location= 'Seoul'
network_type='bike'
trip_times = [5,10,15,20,30] #minutes
travel_speed_avg = (15.5*1000/3600*60)
distance=travel_speed_avg*trip_times[-1]

In [4]:
#get latlong of location
lat,lon = get_latlong(location)

#get network and nearest node
my_network= get_network_from_point((lat,lon),distance,network_type)
center = get_nearest_node(my_network,(lat,lon))


In [5]:
#get city boundary
city = ox.gdf_from_place(location)

#get fast food in seoul
fast_food = ox.pois.pois_from_place(location, amenities=['fast_food'])
fast_food = fast_food[fast_food['cuisine'].notnull()]
fast_food = fast_food[fast_food['name'].notnull()]

#assign icon names
fast_food.loc[fast_food['cuisine'].isin(['doughnut','donut']),'icon']= 'doughnut'
fast_food.loc[fast_food['cuisine'].isin(['burger','sandwich','burger,chicken','sandwiches','hotdogs','american;burger','panini','샌드위치','carb_snack']),'icon'] = 'hamburger' 
fast_food.loc[fast_food['cuisine'].isin(['ice_cream']),'icon'] = 'icecream'
fast_food.loc[fast_food['cuisine'].isin(['chicken','fried_chicken']),'icon'] = 'chicken'
fast_food.loc[fast_food['cuisine'].isin([ 'pizza']),'icon'] = 'pizza'
fast_food.loc[fast_food['cuisine'].isin([ 'mexican']),'icon'] = 'mexican'
fast_food.loc[fast_food['cuisine'].isin([ 'turkish']),'icon'] = 'turkish'
fast_food.loc[fast_food['cuisine'].isin([ 'crepe']),'icon'] = 'crepe'
fast_food.loc[fast_food['cuisine'].isin([ 'bakery']),'icon'] = 'bread'
fast_food.loc[fast_food['cuisine'].isin([ 'asian','korean']),'icon'] = 'asian'

#clip fast food within city boundary
fast_food = fast_food[fast_food.within(city.loc[0, 'geometry'])]

In [78]:
#get destination node
nodes = []
for idx,row in fast_food.iterrows():
    if row['geometry'].geom_type == 'Point' and row['cuisine']== 'korean':
        y,x = [row['geometry'].y,row['geometry'].x]
        nearest_node = get_nearest_node(my_network,(y,x))
        nodes.append(nearest_node)
nodes = list(set(nodes))

#calculate shortest path to any asian restaurant
routes_nodes=[]
for node in nodes:
    route = nx.shortest_path(my_network,source=center,target=node)
    routes_nodes.append(route)
shortest_path_route=  min(routes_nodes, key=len)

#get lat lon of route vertices
route_latlon=[]
for node in shortest_path_route:
    y,x = my_network.node[node]['y'],my_network.node[node]['x']
    route_latlon.append([lat,lon])



In [94]:
#add time attribute to the edges of the network
for u,v,k,data in my_network.edges(data=True,keys=True):
    data['time'] = data['length']/travel_speed_avg
    
# get one color for each isochrone (trip times in minutes)
iso_colors = ox.get_colors(n=len(trip_times), cmap=cm.RdYlGn, start=0,stop=1.0,return_hex=True)

#reproject network for isochrone calculation
my_network= ox.project_graph(my_network,to_crs={'init': 'epsg:3857'})

#get isochrone polygons
isochrone_polys = make_iso_polys(my_network,center, edge_buff=50, node_buff=0, infill=True)

#isochron poly to gdf
data = {'Time':sorted(trip_times,reverse=True)}
df = pd.DataFrame(data)
place_gdf = gpd.GeoDataFrame(df,geometry=isochrone_polys)
place_gdf['hex_color']=iso_colors
place_gdf.crs = {'init': 'epsg:3857'}
place_gdf= place_gdf.to_crs({'init': 'epsg:4326'})

In [173]:
contributor_1 = '<a href="https://www.flaticon.com/authors/smashicons" title="Smashicons" target="_blank">Smashicons</a>'
contributor_2 = '<a href="https://www.flaticon.com/authors/freepik" title="Freepik" target="_blank">Freepik</a>'
contributor_3 = '<a href="https://www.flaticon.com/authors/roundicons" title="Roundicons" target="_blank">Roundicons</a>'
map_author = '<a href="https://github.com/WilliamTjiong" target="_blank">William Tjiong</a>'

credits= '<b>Bike travel time and fast food retaurants in Seoul,South Korea</b> <br>Contributors: &copy<a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a>,&copy<a href=https://www.mapbox.com/about/maps/>Mapbox</a>, &copy<a href="https://carto.com/attributions">CARTO</a><br>Icons: &copy{}, &copy{}, &copy{}<br> Map author: {}'.format(contributor_1,contributor_2,contributor_3,map_author)

m = folium.Map(location=[lat,lon],
               tiles='Cartodb Positron',
               attr=credits,
               zoom_start=12,
              control_scale=True)

folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',name='ESRI Imagery',
                 attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community').add_to(m)
minimap = MiniMap(width=300,height=200,toggle_display=True,zoom_level_offset=-6,tile_layer='Mapbox Bright').add_to(m)

#add cyclist
misc_point = FeatureGroup(name='Hungry bicyclist')
icon_start = CustomIcon(icon_image='./icons/bicycle.png',icon_size=(50,50))
starting_point = folium.Marker((lat,lon),tooltip='A hungry bicyclist',icon=icon_start).add_to(misc_point )
m.add_child(misc_point)

#add fast food restaurants
marker_cluster = MarkerCluster(name='Fast Food Restaurants').add_to(m)
for idx,row in fast_food.iterrows():
    if row['geometry'].geom_type == 'Point':
        latlong = [row['geometry'].y,row['geometry'].x]
        name = row['name']
        cuisine = row['cuisine']
        street= row['addr:street']
        op_hours = row['opening_hours']
        website = row['website']
        text = "<b>Restaurant :</b> {} <br> <b> Dish :</b> {} <br> <b> Street :</b> {} <br> <b> Opening Hours :</b> {}  <br> <b> Website :</b> {}".format(name,cuisine,street,op_hours,website)
        
        icon = CustomIcon(icon_image='./icons/{}.png'.format(row['icon']),icon_size=(30,30))
        fast_food_place = folium.Marker(location=latlong,popup=text,tooltip=text,icon=icon)
        marker_cluster.add_child(fast_food_place)

#add city boundary
city_gjson = city.__geo_interface__
style_city=lambda feature: {
        "fillOpacity": 0,
        'color': '#00bfff',
        'weight': 2
    }
city_feature = folium.features.GeoJson(city_gjson,name='Seoul Administration Boundary',style_function=style_city)
m.add_child(city_feature)

#add isochrone
style_isochrone=lambda feature: {
        'fillColor':feature['properties']['hex_color'],
        "fillOpacity": 0.5,
        'color': 'gray',
        'weight': 0
    }
highlight_isochrone=lambda feature: {
        'color': 'white',
        'weight': 3
    }
isochrone_gjson=place_gdf.__geo_interface__
isochrone_feature=folium.features.GeoJson(isochrone_gjson,name="Bike Travel Time",
                                              style_function=style_isochrone,
                                              highlight_function=highlight_isochrone)
m.add_child(isochrone_feature)

#add route
short_route = FeatureGroup(name='Shortest Route')
food_route=folium.PolyLine(
    locations=route_latlon,
    tooltip='Shortest route to a Korean restaurant',
    weight=2,
    color='black',
    dash_array=[4,4]
).add_to(short_route )
m.add_child(short_route)

#add colorbar
colormap = comap.LinearColormap(
    ['#006837', '#FEFEBD', '#A50026'],
    index=[0,0.5,1.0]
).scale(5,30).to_step(5)
colormap.caption = "Travel time (min)"
colormap.add_to(m)

#add north arrow
url = ('https://raw.githubusercontent.com/SECOORA/static_assets/master/maps/img/rose.png')
FloatImage(url, bottom=78, left=5).add_to(m)


m.add_child(folium.LayerControl(autoZIndex=False, collapsed=False))
m.save('./index.html')